<a href="https://colab.research.google.com/github/JoHussien/DeepLearningCourse/blob/main/WebCrawlerMS2_Databases_YoussefHussien.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# !pip install beautifulsoup4
# !pip install lxml
# !pip install 'h5py==2.10.0' --force-reinstall



In [ ]:
# import the library
import requests
# import the constructor of the parsing thing from the library
from bs4 import BeautifulSoup
import urllib.request
from lxml import etree
import re
import pandas as pd
from urllib.parse import quote
import numpy as np
from datetime import datetime


In [ ]:
#Test crawl the countries in afreica
#[2]
africa = "https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_in_Africa"
asia = "https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_in_Asia"
europe = "https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_in_Europe"
# antarctica = "https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_in_Antarctica"
#[3]
oceania = "https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_in_Oceania"
#[2] and we have an extra table
south_america = "https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_in_South_America"
#[1]
north_america = "https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_in_North_America"


In [ ]:
def get_countries(continent):
  url = continent
  r = requests.get(url)
  soup = BeautifulSoup(r.content,'html5lib')
  countries_names = [] 
  tables = list(soup.find_all(class_ = "wikitable"))
  if continent == south_america:
    # remove last table as it is not of our concern
    tables.pop()
  for table in tables:
    table_rows = list(table.tbody.children)
    filtered_lst = [v for i, v in enumerate(table_rows) if i % 2 == 0]
    table_rows_num = len(filtered_lst) # to define the range of our loop
    # we will store the scraped data here
    index = 2
    if continent == north_america:
      index = 1
    elif continent == oceania:
      index = 3
    for i in range(1,table_rows_num): 
        table_row = filtered_lst[i]
        tds = table_row.find_all("td")
        eng_name = tds[index]
        link = eng_name.find("a")
        eng_name = link.get('title')
        if eng_name:
          countries_names.append(eng_name)
  print("The countires in this continent are: \n",countries_names)
  return countries_names
    

In [ ]:
africa_countries = get_countries(africa)
asia_countries = get_countries(asia)
europe_countries = get_countries(europe)
oceania_countries = get_countries(oceania)
north_america_countries = get_countries(north_america)
south_america_countries = get_countries(south_america)

The countires in this continent are: 
 ['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde', 'Central African Republic', 'Chad', 'Comoros', 'Democratic Republic of the Congo', 'Republic of the Congo', 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon', 'The Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Ivory Coast', 'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe', 'Sahrawi Arab Democratic Republic', 'Somaliland', 'Canary Islands', 'Ceuta', 'French Southern and Antarctic Lands', 'Madeira', 'Mayotte', 'Melilla', 'Pelagie Islands', 'Plazas de soberanía', 'Réunion', 'Socotra Governorate', 'Saint Helena, Ascension and Tristan

In [ ]:
world_countries = africa_countries + asia_countries + europe_countries + oceania_countries + north_america_countries + south_america_countries
print("The world has {} countries.".format(len(world_countries)))

The world has 304 countries.


In [ ]:
af_len  = len(africa_countries)-1
as_len  = len(asia_countries)
eu_len  = len(europe_countries)
oc_len  = len(oceania_countries)
no_len  = len(north_america_countries)
so_len  = len(south_america_countries )
world_cuntries_continents = {"Africa": af_len, "Asia": af_len + as_len, "Europe": af_len + as_len + eu_len, "Oceania": af_len + as_len +  eu_len+ oc_len, "North America": af_len + as_len + oc_len + no_len + eu_len, "South America": af_len + as_len + oc_len + no_len+ so_len+ eu_len}
# world_cuntries_continents

In [ ]:
def get_continent(name):
  dictionary_items = world_cuntries_continents.items()
  try:
    country_position = world_countries.index(name) 
    # print(country_position)
    # print(world_countries[country_position])
  except:
    # print("This country name is not in the original world countries")
    return None
  for item,val in enumerate(dictionary_items):
    # print(val)
    if country_position < val[1]:
      return val[0]

get_continent("Australia")

'Oceania'

In [ ]:
# Now we need to edit the name of each country to match its url
# The only case will be when we have spaces in the country name we replace them with _
# We need to generate url of this format 
#https://en.wikipedia.org/wiki/Country_Name
edited_world_countries = []
# TODO: There is a still a problem in this part with the strange characters
for country in world_countries:
  edited_country_link = "https://en.wikipedia.org/wiki/"
  # We will use a Python defined function to replace whitespaces only with _ if there is none then we don't modify the country name
  # Then we concatenate this to the template url we have
  no_spaces = country.replace(" ", "_")
  edited_country_link += no_spaces
  # edited_country_link.decode()
  # print(edited_country_link)
  edited_world_countries.append(edited_country_link)

print(edited_world_countries[0:10])

['https://en.wikipedia.org/wiki/Algeria', 'https://en.wikipedia.org/wiki/Angola', 'https://en.wikipedia.org/wiki/Benin', 'https://en.wikipedia.org/wiki/Botswana', 'https://en.wikipedia.org/wiki/Burkina_Faso', 'https://en.wikipedia.org/wiki/Burundi', 'https://en.wikipedia.org/wiki/Cameroon', 'https://en.wikipedia.org/wiki/Cape_Verde', 'https://en.wikipedia.org/wiki/Central_African_Republic', 'https://en.wikipedia.org/wiki/Chad']


In [ ]:
# ignore this method for now ... and read the next cell

# Only used it to make you remember that you may want to use regular expressions to match something 
# otherwise this is totally not needed
def make_sure_with_regex_that_this_is_ib_country_largest(class_attribute_value):
    return class_attribute_value and re.compile("ib-.*-largest").search(class_attribute_value)
def make_sure_with_regex_that_this_is_plainlist(class_attribute_value):
    return class_attribute_value and re.compile("plainlist").search(class_attribute_value)

In [ ]:
def get_country_info(id , table_rows):
  table_rows_num = len(table_rows) # to define the range of our loop
  # we will store the scraped data here
  continent = None
  country_name = None     
  capital_name = None
  official_languages = None
  latitude = None
  longitude = None
  driving_side = None
  president = None
  legisilator = None
  gini = None
  currency = None
  calling_code = None
  timezone = None
  hdi = None
  population = None
  area = None
  water = None
  languages = []
  gdp_ppp = None
  gdp_nominal = None
  # of course you can consume the iterator like `for row in table_rows:`
  # but I found it helpful to manipulate the current index of iteration in some situations

  for i in range(table_rows_num): 
      table_row = table_rows[i]
      for table_data in table_row.children:
        # getting president
        # try:
        president_tag1 = table_data.find(text='President')
        if president_tag1:
          # president_names = table_row.find_all('a')
          # president = president_names[1].text
          president = list(table_row.children)[1].text
          # getting name
        # except:
        president_tag2 = table_data.find(text='Monarch')
        if president_tag2:
          president = list(table_row.children)[1].text
        if(president_tag2 is None and president_tag1 is None ):
          res = table_data.find(text='Government')
          if res:
            # print(list(table_rows[i+2].children)[1].text)
            try:
              president = (list(table_rows[i+2].children)[1].text)
              # print(president)
              # print(president)/
            except:
              president = None
          # getting name
        country_name_tag = table_data.find(class_="country-name")
        if country_name_tag:
            country_name = country_name_tag.text
            continent = get_continent(country_name)
        #Getting legisilator
        legislature_tag = table_data.find(text='Legislature')
        if legislature_tag:
          # legisilator_names = table_row.find_all('a')
          # legisilator = legisilator_names[0].text
          # print(list(table_row.children)[1].text)
          legisilator = list(table_row.children)[1].text
        #getting GDP PPP
        gdp_ppp_tag = table_data.find('a', {'title' : 'Purchasing power parity'}) 
        if gdp_ppp_tag:
          # print(list(table_row.children)[1].text.partition('[')[0])
          try:
            gdp_ppp_unedited = (list(table_rows[i+1].children)[1].text)
            gdp_ppp = simplify(gdp_ppp_unedited)
            # print(gdp_ppp)
          except:
            gdp_ppp = None
        #getting GDP nominal
        gdp_nominal_tag = table_data.find(text='(nominal)') 
        if gdp_nominal_tag:
          # print(list(table_row.children)[1].text.partition('[')[0])
          try:
            gdp_nominal_unedited = (list(table_rows[i+1].children)[1].text)
            gdp_nominal = simplify(gdp_nominal_unedited)
            # print(gdp_nominal)
          except:
            gdp_ppp = None
          # print(gini)
        #Getting GDP Nominal
        # Getting Gini Index
        gini_tag = table_data.find('a', {'title' : 'Gini coefficient'}) 
        if gini_tag:
          # print(list(table_row.children)[1].text.partition('[')[0])
          gini_unedited = list(table_row.children)[1].text
          gini = simplify(gini_unedited)
          # print(gini)
        # Getting HDI Index
        hdi_tag = table_data.find('a', {'title' : 'Human Development Index'}) 
        if hdi_tag:
          # print((list(table_row.children)[1].text))
          hdi_unedited = list(table_row.children)[1].text
          # hdi_unedited = hdi_unedited.replace(" ","")
          # print("Hi",hdi_unedited)
          hdi = simplify(hdi_unedited)
          # print(hdi)
        #Getting Curency
        currency_tag = table_data.find(text='Currency')
        if currency_tag:
          # print(table_row.find_all('a')[0].text)
          currency = table_row.find_all('a')[0].text
        # Calling Code
        calling_tag = table_data.find(text='Calling code')
        if calling_tag:
          calling_code = list(table_row.children)[1].text
          # print(calling_code)
        #Getting the TimeZone
        timezone_tag = table_data.find(text='Time zone')
        if timezone_tag:
          timezone = list(table_row.children)[1].text
          # print(timezone)
        #Getting the Official Language(s)
        language_tag = table_data.find(text='Official\xa0languages')
        if language_tag:
          languages_unedited = list(table_row.children)[1].find_all('li')
          if languages_unedited:
            for l in languages_unedited:
              languages.append(l.text)
            # print(languages_unedited)  
          else:
            languages.append(list(table_row.children)[1].text)
        #Getting Driving Side 
        driving_side_tag = table_data.find('a', {'title' : 'Left- and right-hand traffic'}) 
        if driving_side_tag:
          # print(driving_side_tag)
          driving_side = list(table_row.children)[1].text
        #Area 
        area_tag = table_data.find(text='Area ')
        if area_tag:
          #Then we need to go to the next row as this will be the population
          # print(list(table_rows[i+1].children)[1].text)
          try:
            area_unedited = (list(table_rows[i+1].children)[1].text)
            # print(simplify(area_unedited))
            area = simplify(area_unedited)
          except:
            area = None
          # Get water as it is in the same flow
          try:
            water_unedited = (list(table_rows[i+2].children)[1].text)
            # print((water_unedited))
            water = simplify(water_unedited)
            if water > 100: 
              #Then this is no a percentage it is an area of water and we need to get it in terms of percentage
              water = round(water / area * 100,2)
          except:
            water = 0
        #Population
        population_tag = table_data.find(text='Population')
        if population_tag:
          #Then we need to go to the next row as this will be the population
          # print(population_unedited)
          try:
            population_unedited = (list(table_rows[i+1].children)[1].text)
            try:
              population = round(simplify(population_unedited))
            except:
              population = simplify(population_unedited)
          except:
            population = None
        #Capital
        capital_tag = table_data.find(text='Capital')
        if capital_tag:
          # president_names = table_row.find_all('a')
          # president = president_names[1].text
          try:
            # print(table_row)
            capital_name = list(table_row.find_all('a'))[0].text
            # print(capital_name)
          except:
            capital_name = simplify_capital(list(table_row.children)[1].text)
        if capital_name is None:
          capital_tag = table_data.find(text='Capital city')
          if capital_tag:
            capital_name = simplify_capital(list(table_row.children)[1].text)
            # print((capital_name))
        # capital_tag = table_data.find(class_=make_sure_with_regex_that_this_is_ib_country_largest)
        # # print(capital_tag)
        # # See the if condition again so we don't overwrite the data
        # if capital_tag:
        #   # print(capital_tag)
        #   second_child = list(table_row.children)[1]
        #   capital_name_tag = second_child.find(title=True)
        #   if capital_name_tag:
        #       capital_name = capital_name_tag.text
        #   if second_child.find(class_="latitude") :
        #     latitude = second_child.find(class_="latitude").text
        #   if second_child.find(class_="longitude"):
        #     longitude = second_child.find(class_="longitude").text
  # #Handling edge case of president
 

  return [id, continent, country_name, capital_name, driving_side,simplify_capital(president), legisilator,gdp_ppp,gdp_nominal, gini,hdi, currency, calling_code,timezone,area,water,population, languages]
  


In [ ]:
def simplify(unedited):
  # First we need to remvoe all space
  # new_unedited = unedited.replace(" ", "")
  new_unedited = unedited.strip()
  final = ''
  # print(new_unedited)
  for char in new_unedited:
    if char.isdigit() or char=='.' or char==',' or char=='$' or char =='€':
      # Skip this symbol as it cannot be converted to float
      if char != ',' and char != '$' and char != '€':
        final+= char
    else:
      break
  # print(final)
  try:
    result = float(final)
    return result
  except:
    return new_unedited



In [ ]:
def simplify_capital(unedited):
  final = ''
  if unedited is not None:
    for char in unedited:
      if char ==' ' or char=='_' or char=='-':
        final+=char
      elif char.isdigit() or not char.isalpha() :
        break
        
      else:
        final+=char
  # print(final)
  return final



In [ ]:
print(simplify_capital("hello0]"))

hello


In [ ]:
edited_world_countries[302]

'https://en.wikipedia.org/wiki/South_Georgia_and_the_South_Sandwich_Islands'

In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/Egypt")
soup = BeautifulSoup(page.content)
table_rows = []
table_rows = list(soup.find(class_ = "infobox").tbody.children)
country_info = get_country_info(1 , table_rows) 

In [ ]:
print(country_info)

[1, None, 'Arab Republic of Egypt', 'Cairo', 'right', 'Abdel Fattah el-Sisi', 'Parliament', 1.391, 362.0, 31.5, 0.707, 'Egyptian pound', '+20', 'UTC+2[c] (EGY)', 1010408.0, 0.632, 101478581, ['Arabic']]


In [ ]:
final_data = []
id = 1
for country_url in edited_world_countries:
  # print(country_url)
  soup = None
  while(soup is None):
    page = requests.get(country_url)
    soup = BeautifulSoup(page.content)

  table_rows = []
  table_rows = list(soup.find(class_ = "infobox").tbody.children)
  if(country_url!= "https://en.wikipedia.org/wiki/Chile"):
    country_info = get_country_info(id , table_rows)
    # This is only done to the continent
    if country_info[1] is None:
      country_info[1] = get_continent(world_countries[id - 1])
    # This is only done to the country name
    if country_info[2] is None:
      country_info[2] = world_countries[id - 1]
  final_data.append(country_info)
  id+=1
  print(id)
  # if id == 30:
  #   break


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [ ]:
final_data_np = (np.array(final_data))
headers = ["id","Continent" ,"Country Name", "Capital", "Driving Side", "President", "Legislature","GDP PPP ($ Billion)","GDP Nominal ($ Billion)" ,"Gini Index", "HDI Index", "Currency", "Calling Code","Time Zone","Area (km^2)", "Water %","Population", "Official Languages"]
final_data_dataframe = pd.DataFrame(final_data_np, index = final_data_np[:,0])
final_data_dataframe.columns = headers
final_data_dataframe

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


,id,Continent,Country Name,Capital,Driving Side,President,Legislature,GDP PPP ($ Billion),GDP Nominal ($ Billion),Gini Index,HDI Index,Currency,Calling Code,Time Zone,Area (km^2),Water %,Population,Official Languages
1,1,Africa,People's Democratic Republic of Algeria,Algiers,right,Abdelmadjid Tebboune,Parliament,532.565,163512,27.6,0.748,Algerian dinar,+213,UTC+1 (CET),2.38174e+06,1.1,44700000,"[Arabic, Berber]"
2,2,Africa,Republic of Angola,Luanda,right,João Lourenço,National Assembly,213.034,64.48,51.3,0.581,Kwanza,+244,UTC+1 (WAT),1.2467e+06,0,31127674,[Portuguese]
3,3,Africa,Republic of Benin,Porto-Novo,right,Patrice Talon,National Assembly,29.918,11.386,47.8,0.545,West African CFA franc,+229,UTC+1 (WAT),114763,0.4,11733059,[French]
4,4,Africa,Republic of Botswana,Gaborone,left,Mokgweetsi Masisi,National Assembly,43.389,18.726,53.3,0.735,Pula,+267,Central Africa Time (GMT+2)[15],581730,2.7,2254068,"[Setswana, English]"
5,5,Africa,Burkina Faso,Ouagadougou,right,Roch Marc Christian Kaboré,National Assembly,45.339,16.226,38.9,0.452,West African CFA franc,+226,UTC (GMT),274200,0.146,21510181,[French]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,300,South America,Bolivarian Republic of Venezuela,Caracas,right,Nicolás Maduro,National Assembly,142.416,44.893,44.8,0.711,Bolívar digital,+58,UTC−4 (VET),916445,3.2,28887118,[Spanish[b]]
301,301,South America,Bouvet Island,None,None,Harald,None,None,None,None,None,None,None,None,None,None,0,[]
302,302,South America,Falkland Islands,Stanley,left,Elizabeth II,Legislative Assembly,228.5,None,36,0.874,Falkland Islands pound,+500,UTC-03:00 (FKST),None,None,3398,[]
303,303,South America,South Georgia and the South Sandwich Islands,King Edward Point,left,Elizabeth II,None,None,None,None,None,Falkland Islands pound,+500,UTC−02:00,None,None,None,[]


In [ ]:
final_data_dataframe.loc[[220]]

,id,Continent,Country Name,Capital,Driving Side,President,Legislature,GDP PPP ($ Billion),GDP Nominal ($ Billion),Gini Index,HDI Index,Currency,Calling Code,Time Zone,Area (km^2),Water %,Population,Official Languages
220,220,Oceania,Galápagos Islands,Puerto Baquerizo Moreno,None,,None,None,None,None,None,None,None,GALT (UTC-6),None,None,3,[]


In [ ]:
final_data_dataframe.to_csv('Countries_data.csv',index=False,encoding='utf-8-sig')
from google.colab import files
files.download("Countries_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
final_data_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 304 entries, 1 to 304
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       304 non-null    object
 1   Continent                303 non-null    object
 2   Country Name             304 non-null    object
 3   Capital                  268 non-null    object
 4   Driving Side             261 non-null    object
 5   President                304 non-null    object
 6   Legislature              252 non-null    object
 7   GDP PPP ($ Billion)      234 non-null    object
 8   GDP Nominal ($ Billion)  251 non-null    object
 9   Gini Index               189 non-null    object
 10  HDI Index                225 non-null    object
 11  Currency                 271 non-null    object
 12  Calling Code             265 non-null    object
 13  Time Zone                295 non-null    object
 14  Area (km^2)              212 non-null    o

In [ ]:
# edited_world_countries[282]

In [ ]:
# bool_series = pd.isnull(final_data_dataframe["President"]) 
    
# # filtering data 
# # displaying data only with Gender = NaN 
# list(final_data_dataframe[bool_series]['id'].index)

For each president / monarch, you need to store his/her name, birthdate, the date he/she assumed office, and his/her political party. You also need to store information about capital city for each country, including its name, population, area / metro area, governor, and coordinates.

In [ ]:
presidents = final_data_dataframe[['id','President']]
presidents

,id,President
1,1,Abdelmadjid Tebboune
2,2,João Lourenço
3,3,Patrice Talon
4,4,Mokgweetsi Masisi
5,5,Roch Marc Christian Kaboré
...,...,...
300,300,Nicolás Maduro
301,301,Harald
302,302,Elizabeth II
303,303,Elizabeth II


In [ ]:
# Now we need to edit the name of each country to match its url
# The only case will be when we have spaces in the country name we replace them with _
# We need to generate url of this format 
#https://en.wikipedia.org/wiki/Country_Name
edited_presidents = []
# TODO: There is a still a problem in this part with the strange characters
for president in presidents['President']:
  edited_president_link = "https://en.wikipedia.org/wiki/"
  # We will use a Python defined function to replace whitespaces only with _ if there is none then we don't modify the country name
  # Then we concatenate this to the template url we have
  try:
    no_spaces = president.replace(" ", "_")
    edited_president_link += no_spaces
    edited_presidents.append(edited_president_link)
  except:
    edited_presidents.append('')


# print(edited_presidents[0:20])

In [ ]:
def trunc_at(s, d, n=3):
    "Returns s truncated at the n'th (3rd by default) occurrence of the delimiter, d."
    res =  d.join(s.split(d, n)[:n])
    res_copy = res
    for i in range(len(res)-1,1,-1):
      if res_copy[i].isalpha():
        res = res[:-1]
      else:
        break
    return res
print(trunc_at("5 March 2013Acting until 19 April 2013Disputed with Juan Guaidósince 23 January 2019", " "))

5 March 2013


In [ ]:
def get_president_info(table_rows):
  table_rows_num = len(table_rows) # to define the range of our loop
  # we will store the scraped data here population, area / metro area, governor, and coordinates
  BD = None
  officeDate = None
  politicalParty = []
  # of course you can consume the iterator like `for row in table_rows:`
  # but I found it helpful to manipulate the current index of iteration in some situations
  first = True
  for i in range(table_rows_num): 
      table_row = table_rows[i]
      for table_data in table_row.children:
        # getting BirthDate
        BD_tag = table_data.find(text='Born')
        if BD_tag:
          try:
            BD_unedited = table_row.find_all('span',{'class':'bday'})
            BD = BD_unedited[0].text
          except:
            BD = int(simplify(list(table_row.children)[1].text))
        # getting Office Date
        office_tag = table_data.find(text='Assumed office')
        if office_tag:
          if first:
            try:
              office_unedited = table_data.text
              # print(office_unedited,"id")
              date_unedited = office_unedited.replace("Assumed office ",'')
              date_edited = trunc_at(date_unedited,' ')
              date_date = datetime.strptime(date_edited, '%d %B %Y')
              date_string = datetime.strftime(date_date,'%Y-%m-%d')
              officeDate = date_string
              # print(officeDate)
            except:
              #TODO FIND ANOTHER WAY WHEN IT IS NOT SPAN AS IN Évariste_Ndayishimiye
              officeDate = None 
          first = False
        #Getting the Political Party
        politicalParty_tag = table_data.find(text='Political party')
        if politicalParty_tag:
          politicalParty_unedited = list(table_row.children)[1].find_all('li')
          if politicalParty_unedited:
            for l in politicalParty_unedited:
              politicalParty.append(l.text)
            # print(languages_unedited)  
          else:
            politicalParty.append(list(table_row.children)[1].text)
        
  return [BD, officeDate, politicalParty]
  



In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/Nicolás_Maduro")
soup = BeautifulSoup(page.content)
table_rows = []
table_rows = list(soup.find(class_ = "infobox").tbody.children)
country_info = get_president_info(table_rows) 
country_info


['1962-11-23',
 '2013-03-05',
 ['United Socialist Party of Venezuela (from 2007)Fifth Republic Movement (before 2007)']]

In [ ]:
president_data_additional = []
local_id = 1
for president_url in edited_presidents:
  # print(president_url)
  try:
    soup = None
    # while(soup is None):
    page = requests.get(president_url)
    soup = BeautifulSoup(page.content)

    table_rows = []
    table_rows = list(soup.find(class_ = "infobox").tbody.children)
    president_info = get_president_info(table_rows)
    president_data_additional.append(president_info)
  except:
    president_data_additional.append([None,None,None])
  local_id+=1
  print(local_id)
  # if local_id == 10:
  #   break
# president_data_additional

In [ ]:
president_data_additional_np = (np.array(president_data_additional))
headers = ["Birthdate", "Office Date", "Political Party"]
presidents_data_additional_dataframe = pd.DataFrame(president_data_additional_np)
presidents_data_additional_dataframe.columns = headers
presidents_data_additional_dataframe.index = presidents_data_additional_dataframe.index + 1
presidents_data_additional_dataframe['id']=presidents_data_additional_dataframe.index
# presidents_data_additional_dataframe

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
presidents_data_all = pd.merge(on='id', left = presidents, right=presidents_data_additional_dataframe)
presidents_data_all


,id,President,Birthdate,Office Date,Political Party
0,1,Abdelmadjid Tebboune,1945-11-17,2019-12-19,[National Liberation Front]
1,2,João Lourenço,1954-03-05,2017-09-26,[People's Movement for the Liberation of Angol...
2,3,Patrice Talon,1958-05-01,2016-04-06,[Independent]
3,4,Mokgweetsi Masisi,1962-07-21,2018-04-01,[Botswana Democratic Party]
4,5,Roch Marc Christian Kaboré,1957-04-25,2015-12-29,[Organization for Popular Democracy – Labour M...
...,...,...,...,...,...
299,300,Nicolás Maduro,1962-11-23,2013-03-05,[United Socialist Party of Venezuela (from 200...
300,301,Harald,None,None,None
301,302,Elizabeth II,1926-04-21,None,[]
302,303,Elizabeth II,1926-04-21,None,[]


In [ ]:
presidents_data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 304 entries, 0 to 303
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               304 non-null    object
 1   President        304 non-null    object
 2   Birthdate        246 non-null    object
 3   Office Date      164 non-null    object
 4   Political Party  250 non-null    object
dtypes: object(5)
memory usage: 14.2+ KB


In [ ]:
get_null = pd.isna(presidents_data_all['Office Date'])
presidents_data_all[get_null].head(30)

,id,President,Birthdate,Office Date,Political Party
7,8,José Maria Neves,1960-03-28,None,[African Party for the Independence of Cape Ve...
17,18,Mswati III,1968-04-19,None,[]
20,21,Adama Barrow,1965-02-15,None,[National People's Party (2019–present)[6][7]]
22,23,Mamady Doumbouya,None,None,None
26,27,Letsie III,1963-07-17,None,[]
31,32,Assimi Goïta,None,None,None
34,35,Mohammed VI,None,None,None
40,41,Carlos Vila Nova,None,None,[ADI]
45,46,Cyril Ramaphosa,1952-11-17,None,[African National Congress]
57,58,Council of Government,None,None,None


In [ ]:
presidents_data_all.to_csv('Presidents_data.csv',index=False,encoding='utf-8-sig')
from google.colab import files
files.download("Presidents_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
countries_presidents_data_all = pd.merge(on='id', left = final_data_dataframe, right=presidents_data_all)
countries_presidents_data_all = countries_presidents_data_all.drop(['President_x'], axis=1)
countries_presidents_data_all = countries_presidents_data_all.rename(columns={'President_y': 'President'})
countries_presidents_data_all

In [ ]:
countries_presidents_data_all.to_csv('Countries_Presidents_data.csv',index=False,encoding='utf-8-sig')
from google.colab import files
files.download("Countries_Presidents_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



---



---



In [ ]:
capitals = final_data_dataframe[['id','Capital']]
capitals

,id,Capital
1,1,Algiers
2,2,Luanda
3,3,Porto-Novo
4,4,Gaborone
5,5,Ouagadougou
...,...,...
300,300,Caracas
301,301,None
302,302,Stanley
303,303,King Edward Point


In [ ]:
# Now we need to edit the name of each country to match its url
# The only case will be when we have spaces in the country name we replace them with _
# We need to generate url of this format 
#https://en.wikipedia.org/wiki/Country_Name
edited_capitals = []
# TODO: There is a still a problem in this part with the strange characters
for capital in capitals['Capital']:
  edited_capital_link = "https://en.wikipedia.org/wiki/"
  # We will use a Python defined function to replace whitespaces only with _ if there is none then we don't modify the country name
  # Then we concatenate this to the template url we have
  try:
    no_spaces = capital.replace(" ", "_")
    edited_capital_link += no_spaces
    edited_capitals.append(edited_capital_link)
  except:
    edited_capitals.append('')


print(edited_capitals[0:20])

['https://en.wikipedia.org/wiki/Algiers', 'https://en.wikipedia.org/wiki/Luanda', 'https://en.wikipedia.org/wiki/Porto-Novo', 'https://en.wikipedia.org/wiki/Gaborone', 'https://en.wikipedia.org/wiki/Ouagadougou', 'https://en.wikipedia.org/wiki/Gitega', 'https://en.wikipedia.org/wiki/Yaoundé', 'https://en.wikipedia.org/wiki/Praia', 'https://en.wikipedia.org/wiki/Bangui', "https://en.wikipedia.org/wiki/N'Djamena", 'https://en.wikipedia.org/wiki/Moroni', 'https://en.wikipedia.org/wiki/Kinshasa', 'https://en.wikipedia.org/wiki/Brazzaville', 'https://en.wikipedia.org/wiki/Djibouti', 'https://en.wikipedia.org/wiki/Cairo', 'https://en.wikipedia.org/wiki/Malabo', 'https://en.wikipedia.org/wiki/Asmara', 'https://en.wikipedia.org/wiki/Mbabane', 'https://en.wikipedia.org/wiki/Addis_Ababa', 'https://en.wikipedia.org/wiki/Libreville']


In [ ]:
def get_capital_info(table_rows):
  table_rows_num = len(table_rows) # to define the range of our loop
  # we will store the scraped data here

  population = None
  area = None
  governor = None
  longitude = None
  latitude = None
  # of course you can consume the iterator like `for row in table_rows:`
  # but I found it helpful to manipulate the current index of iteration in some situations

  for i in range(table_rows_num): 
      table_row = table_rows[i]
      for table_data in table_row.children:
        #getting longitude
        longitude_tag = table_data.find(class_='longitude')
        if longitude_tag:
          longitude_unedited = table_data.find('span',{'class':'longitude'}).text
          longitude = longitude_unedited
          
        #getting latitude
        latitude_tag = table_data.find(class_='latitude')
        if latitude_tag:
          latitude_unedited = table_data.find('span',{'class':'latitude'}).text
          latitude = latitude_unedited
        #getting population
        population_tag = table_data.find(text='Population')
        if population_tag:
          try:
            population_unedited = list(table_rows[i+1].children)[1].text
            if population_unedited:
              population = int(simplify(population_unedited))
          except:
            population = None
        #getting Area (Capital City)
        area_tag = table_data.find(text='Area')
        if area_tag:
          try:
            area_unedited = list(table_rows[i+1].children)[1].text
            if area_unedited:
              area = (simplify(area_unedited))
          except:
            area = None
       #Getting the governor
        res = table_data.find(text='Government')
        if res:
          # print(list(table_rows[i+2].children)[1].text)
          try:
            governor = simplify_capital(list(table_rows[i+1].children)[1].text)
            # print((governor))
            # print(president)/
          except:
            governor = None     
        # governor_tag1 = table_data.find(text='Governor')
        # if governor_tag1:
        #   governor1 = list(table_row.children)[1].text
          # if governorer == 'Mayor':
          #   governorer = governor1
          # print(governor, " h")
          # getting name
        # except:
        governor_tag = table_data.find(text='Mayor')
        if governor_tag:
          governorer1 = list(table_row.children)[1].text
          
          # print(governor," hh")
        # if(governor_tag2 is None and governor_tag1 is None ):
  if governor == 'Mayor':
            governor = governor1          
  return [population,area,governor,longitude,latitude]
  



In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/Porto-Novo")
soup = BeautifulSoup(page.content)
table_rows = []
table_rows = list(soup.find(class_ = "infobox").tbody.children)
country_info = get_capital_info(table_rows) 
country_info

[264320, 110.0, 'Emmanuel Zossou', '2°36′18″E', '6°29′50″N']

In [ ]:
capital_data_additional = []
local_id = 1
for capital_url in edited_capitals:
  # print(capital_url)
  try:
    soup = None
    # while(soup is None):
    page = requests.get(capital_url)
    soup = BeautifulSoup(page.content)

    table_rows = []
    table_rows = list(soup.find(class_ = "infobox").tbody.children)
    capital_info = get_capital_info(table_rows)
    capital_data_additional.append(capital_info)
  except:
    capital_data_additional.append([None,None,None,None,None])
  local_id+=1
  print(local_id)
  # if local_id == 10:
  #   break
capital_data_additional

In [ ]:
capital_data_additional_np = (np.array(capital_data_additional))
headers = ["Population", "Area (Capital City km^2)", "Governor","Longitude","Latitude"]
capital_data_additional_dataframe = pd.DataFrame(capital_data_additional_np)
capital_data_additional_dataframe.columns = headers
capital_data_additional_dataframe.index = capital_data_additional_dataframe.index + 1
capital_data_additional_dataframe['id']=capital_data_additional_dataframe.index
capital_data_additional_dataframe

,Population,Area (Capital City km^2),Governor,Longitude,Latitude,id
1,3915811,363,Abdelkader Seyouda,3°3′32″E,36°45′14″N,1
2,2571861,116,None,13°14′4″E,8°50′18″S,2
3,264320,110,Emmanuel Zossou,2°36′18″E,6°29′50″N,3
4,231626,169,City commission government,25°54′44″E,24°39′29″S,4
5,2453496,520,Armand Béouindé,1°32′W,12°22′N,5
...,...,...,...,...,...,...
300,None,None,None,None,None,300
301,None,None,None,None,None,301
302,None,None,None,None,None,302
303,22,None,None,36°30′00″W,54°17′00″S,303


In [ ]:
# capitals.loc[[303]]


In [ ]:
capitals_data_all = pd.merge(on='id', left = capitals, right=capital_data_additional_dataframe)
capitals_data_all
# presidents_data_all = pd.concat([presidents, presidents_data_additional_dataframe],axis=1)
# presidents_data_all

,id,Capital,Population,Area (Capital City km^2),Governor,Longitude,Latitude
0,1,Algiers,3915811,363,Abdelkader Seyouda,3°3′32″E,36°45′14″N
1,2,Luanda,2571861,116,None,13°14′4″E,8°50′18″S
2,3,Porto-Novo,264320,110,Emmanuel Zossou,2°36′18″E,6°29′50″N
3,4,Gaborone,231626,169,City commission government,25°54′44″E,24°39′29″S
4,5,Ouagadougou,2453496,520,Armand Béouindé,1°32′W,12°22′N
...,...,...,...,...,...,...,...
299,300,Caracas,None,None,None,None,None
300,301,None,None,None,None,None,None
301,302,Stanley,None,None,None,None,None
302,303,King Edward Point,22,None,None,36°30′00″W,54°17′00″S


In [ ]:
capitals_data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 304 entries, 0 to 303
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        304 non-null    object
 1   Capital                   268 non-null    object
 2   Population                206 non-null    object
 3   Area (Capital City km^2)  190 non-null    object
 4   Governor                  158 non-null    object
 5   Longitude                 209 non-null    object
 6   Latitude                  209 non-null    object
dtypes: object(7)
memory usage: 19.0+ KB


In [ ]:
get_null = pd.isna(capitals_data_all['Governor'])
capitals_data_all[get_null].head(30)

,id,Capital,Population,Area (Capital City km^2),Governor,Longitude,Latitude
1,2,Luanda,2571861,116,None,13°14′4″E,8°50′18″S
5,6,Gitega,135467,22,None,29°54′E,3°26′S
6,7,Yaoundé,2765600,180,None,11°31′E,3°52′N
7,8,Praia,159050,102.6,None,23°30′32″W,14°55′05″N
9,10,N'Djamena,916000,104,None,15°03′00″E,12°06′36″N
10,11,Moroni,None,None,None,None,None
13,14,Djibouti,921804,None,None,43°10′E,11°36′N
15,16,Malabo,297000,21,None,8°46′25.32″E,3°45′7.43″N
19,20,Libreville,703904,65.42,None,9°27′15″E,0°23′25″N
21,22,Accra,5055900,173,None,0°12′W,5°33′N


In [ ]:
capitals_data_all.to_csv('Capitals_data.csv',index=False,encoding='utf-8-sig')
from google.colab import files
files.download("Capitals_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Generating Fake Data

In [ ]:
import pandas as pd
from datetime import date
def fake_customer_data(n):
  customers_data = []
  for i in range(n):
    email = faker.email()
    username = faker.user_name()
    date = faker.date_time().strftime('%Y-%m-%d')
    gender = faker.profile()['sex']
    DOB = faker.profile()['birthdate']
    today_date_ = datetime.today().strftime('%Y-%m-%d')
    today_date = datetime.date(datetime.strptime(today_date_,'%Y-%m-%d'))
    age = today_date - DOB
    age = round(age.days/365,0)
    customers_data.append([email,username,date,gender,age])
  headers = ['Email','UserName','Date','Gender','Age']
  customers_pd = pd.DataFrame(customers_data)
  customers_pd.columns = headers
  return customers_pd

fake_customers = fake_customer_data(20)
print(fake_customers)
fake_customers.to_csv('Customers_data.csv',index=False,encoding='utf-8-sig')
from google.colab import files
files.download("Customers_data.csv")



---



---

